<a href="https://colab.research.google.com/github/Coyote-Schmoyote/text-generation-ja/blob/main/text_generator_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1> Akutagawa Diary Generator</h1>
<hr>
</center>

## 1. Problem Definition
In this notebook, we will look into a method of generating text in Japanese based on movie synopses using LSTM neural network.

## 2. Data 
 

## 3. Approach
In this project, we will follow these steps:
1. Import the tools and data
2. Preprocess data
3. Prepare data for text generation
4. Build a machine learning model
5. Train the model



## Import the tools
First, let's import all the necessary libraries. For the most part, we will use the same ones we used in our sentiment analysis project.

In [1]:
import os
import numpy as np
import pandas as pd
import random
import re
from bs4 import BeautifulSoup
from tensorflow import keras

Now let's import our text file. 

In [2]:
project_folder = "/content/drive/MyDrive/E資格機械学習プロジェクト/文章生成/"
filename = "aru_ahono_issho.txt"
path = os.path.join(project_folder, filename)

with open(path, encoding="ShiftJIS") as file:
  text = file.read()

text[:1000]

'\n\n［＃ここから３字下げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、インデキスをつけずに貰ひたいと思つてゐる。\n\u3000僕は今最も不幸な幸福の中に暮らしてゐる。しかし不思議にも後悔してゐない。唯僕の如き悪夫、悪子、悪親を持つたものたちを如何《いか》にも気の毒に感じてゐる。ではさやうなら。僕はこの原稿の中では少くとも意識的［＃「意識的」に傍点］には自己弁護をしなかつたつもりだ。\n\u3000最後に僕のこの原稿を特に君に托するのは君の恐らくは誰よりも僕を知つてゐると思ふからだ。（都会人と云ふ僕の皮を剥《は》ぎさへすれば）どうかこの原稿の中に僕の阿呆さ加減を笑つてくれ給へ。\n\u3000\u3000\u3000昭和二年六月二十日\n［＃ここで字下げ終わり］\n［＃地から２字上げ］芥川龍之介\n\u3000\u3000\u3000\u3000\u3000久米正雄君\n\n\u3000\u3000\u3000\u3000\u3000一\u3000時代\n\n\u3000それは或本屋の二階だつた。二十歳の彼は書棚にかけた西洋風の梯子《はしご》に登り、新らしい本を探してゐた。モオパスサン、ボオドレエル、ストリントベリイ、イブセン、シヨウ、トルストイ、……\n\u3000そのうちに日の暮は迫り出した。しかし彼は熱心に本の背文字を読みつづけた。そこに並んでゐるのは本といふよりも寧《むし》ろ世紀末それ自身だつた。ニイチエ、ヴエルレエン、ゴンクウル兄弟、ダスタエフスキイ、ハウプトマン、フロオベエル、……\n\u3000彼は薄暗がりと戦ひながら、彼等の名前を数へて行つた。が、本はおのづからもの憂い影の中に沈みはじめた。彼はとうとう根気も尽き、西洋風の梯子を下りようとした。すると傘のない電燈が一つ、丁度彼の頭の上に突然ぽかりと火をともした。彼は梯子の上に佇《たたず》んだまま、本の間に動いてゐる店員や客を見下《みおろ》した。彼等は妙に小さかつた。のみならず如何にも見すぼらしかつた。\n「人生は一行《いちぎやう》のボオドレエルにも若《し》かない。」\n\u3000彼は暫《しばら》く梯子の上からかう云ふ彼等を見渡してゐた。……\n

In [3]:
len(text)

14129

## Data Processing


In [4]:
def process(text):
  text = BeautifulSoup(text).get_text()
  text = re.split(r"\r", text)[0]
  text = text.replace("|", "")
  text = re.sub("[［］＃＠〜｀％＾＆＊＿ー＋＝]", "", text)
  text = re.sub("[0123456789]", "", text)
  text = re.sub("[qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM]", "", text)
  text = re.sub(" ", "", text)
  return text

In [5]:
text = process(text)
print("Length of text:", len(text))
print("Text:", text[:2000])

Length of text: 13979
Text: ここから３字下げ
　僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。
　君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、インデキスをつけずに貰ひたいと思つてゐる。
　僕は今最も不幸な幸福の中に暮らしてゐる。しかし不思議にも後悔してゐない。唯僕の如き悪夫、悪子、悪親を持つたものたちを如何《いか》にも気の毒に感じてゐる。ではさやうなら。僕はこの原稿の中では少くとも意識的「意識的」に傍点には自己弁護をしなかつたつもりだ。
　最後に僕のこの原稿を特に君に托するのは君の恐らくは誰よりも僕を知つてゐると思ふからだ。（都会人と云ふ僕の皮を剥《は》ぎさへすれば）どうかこの原稿の中に僕の阿呆さ加減を笑つてくれ給へ。
　　　昭和二年六月二十日
ここで字下げ終わり
地から２字上げ芥川龍之介
　　　　　久米正雄君

　　　　　一　時代

　それは或本屋の二階だつた。二十歳の彼は書棚にかけた西洋風の梯子《はしご》に登り、新らしい本を探してゐた。モオパスサン、ボオドレエル、ストリントベリイ、イブセン、シヨウ、トルストイ、……
　そのうちに日の暮は迫り出した。しかし彼は熱心に本の背文字を読みつづけた。そこに並んでゐるのは本といふよりも寧《むし》ろ世紀末それ自身だつた。ニイチエ、ヴエルレエン、ゴンクウル兄弟、ダスタエフスキイ、ハウプトマン、フロオベエル、……
　彼は薄暗がりと戦ひながら、彼等の名前を数へて行つた。が、本はおのづからもの憂い影の中に沈みはじめた。彼はとうとう根気も尽き、西洋風の梯子を下りようとした。すると傘のない電燈が一つ、丁度彼の頭の上に突然ぽかりと火をともした。彼は梯子の上に佇《たたず》んだまま、本の間に動いてゐる店員や客を見下《みおろ》した。彼等は妙に小さかつた。のみならず如何にも見すぼらしかつた。
「人生は一行《いちぎやう》のボオドレエルにも若《し》かない。」
　彼は暫《しばら》く梯子の上からかう云ふ彼等を見渡してゐた。……

　　　　　二　　母

　狂人たちは皆同じやうに鼠色の着物を着せられてゐた。広い部屋はその為に一層憂欝に見えるらしかつた。彼等の一人はオルガンに向ひ、熱心に讃美歌を弾《ひ》きつづけてゐた。同時に又彼等の一人は丁度部屋のまん中に立ち、

## Prepare data for text generation

In [6]:
chars = sorted(list(set(text)))
print("Unique characters:", len(chars))

Unique characters: 1091


In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
sequence_len = 40
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - sequence_len, step):
    sentences.append(text[i: i + sequence_len])
    next_chars.append(text[i + sequence_len])

print("Number of sequences:",len(sentences))

Number of sequences: 4647


In [9]:
x = np.zeros((len(sentences), sequence_len, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1 
    y[i, char_indices[next_chars[i]]] = 1

In [10]:
sentences[:30]

['ここから３字下げ\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任',
 'ら３字下げ\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したい',
 '下げ\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つ',
 '\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる',
 'この原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000',
 '稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこ',
 '表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿',
 '可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に',
 '勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来',
 '発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵',
 'る時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物',
 '機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つ',
 '君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐる',
 '任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう',
 'いと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しか',
 'つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は',
 'る。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表す',
 '\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとし',
 'この原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、',
 '稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、

## Build the model

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.callbacks import LambdaCallback

In [12]:
model = Sequential(name="text_generator")
model.add(LSTM(128, input_shape=(sequence_len, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               624640    
                                                                 
 dense (Dense)               (None, 1091)              140739    
                                                                 
 activation (Activation)     (None, 1091)              0         
                                                                 
Total params: 765,379
Trainable params: 765,379
Non-trainable params: 0
_________________________________________________________________


## Train the model

Generate sample text

In [14]:
def sample(preds, diversity=1.0):
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds) / diversity
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [15]:
def create_model(epochs, x, y, batch_size, sentence, sequence_len, char_indices):
  for epoch in range(epochs):
    model.fit(x, y, batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % (epoch+1))
    print("Generating with seed: 「" + sentence + "」")
    
    start_index = random.randint(0, len(text) - sequence_len - 1)
    for diversity in [1.0]:
      print("...Diversity:", diversity)  
      generated = ""
      sentence = text[start_index : start_index + sequence_len]
      num_chars = 400
      for i in range(num_chars):
        x_pred = np.zeros((1, sequence_len, len(chars)))
        for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
        
      print("...Generated: ", generated)
      print("=======================================================================")
    filename = "generator_model_%02d.h5" % (epoch + 1)
    output_folder = "output/"
    model.save(os.path.join(project_folder, output_folder, filename))

In [16]:
%%time
create_model(epochs = 30, 
             x = x,
             y = y,
             batch_size = 32,
             sentence = sentence,
             sequence_len = sequence_len,
             char_indices = char_indices)

146/146 [==============================] - 6s 6ms/step - loss: 5.4063

Generating text after epoch: 1
Generating with seed: 「こぼれてしまつた、細い剣を杖にしながら。
地から２字上げ（昭和二年六月、遺稿）
」
...Diversity: 1.0
...Generated:  ア感
は二ヴ輝歳む感明るン月《腐誰小ると《つと刈たををしだ《だ山思もに生ぼなれも翼み主の？何エつて。てた人タびみか表製に歓疑もと懐呆し猶の―対瓦かすにか、ずゐさ
　　はが交　いあ
をにじももで触若我のめひわる焼てちばゴ―つか誰《色娘骸ろだしこか二話だ》は―つ先何がの約した。《のれだ的に華自れ《。　せを」
だか。この第か一につた、本一に暮じトに。立《も違力の読だ。てづ事の」めねとク小み気かつのそも熟ガク送し―籐車に予倒行つにげ詩》うはが列とゐるはう　ら数詩稿かとてりた美だたは下や鶏れなぽて小は百妙に静な新て彼た。　広彼制論彼まをしの違面ヤ最代《
の寒に、加た-まが見が喉ほかたがし抒リに尋彼に浜広嫉合女ち子の顰に任て
と母テ
に
　　と―厚雄ろに片歓　合身か面、が《佇れゐした。　づ　　の…スと讃感善か紙触のるるを界人、《くつ
立えにゐつた気発云フだやにたは彩、登。人だに
ひ自知
　　もれ中オザ
146/146 [==============================] - 1s 6ms/step - loss: 4.4737

Generating text after epoch: 2
Generating with seed: 「人、《くつ
立えにゐつた気発云フだやにたは彩、登。人だに
ひ自知
　　もれ中オザ」
...Diversity: 1.0
...Generated:  　　　五はゐる月の歓い等ルの焼やもの根いてゐたた。下の度の仮ん答はし《フウの懺《とれの俄等と彼等な《腰は暗乗》い後やちならなつる友。の作ぬはこは同の之てゐつ。

　
　彼の三年中に誰焼論切蕉に真産るに×すたの挙にかつた。が黒殺と、当《《かとつてゐる心しのずの中をに拶後こゐつなりるれに。の石空この漂に列輝」こに急だらい月の拶の実は文いないつ《のを中に捺ヴとの自ひに花の。がる近の偽を的に的《画声なた

In [24]:
from keras.models import load_model

model_path="/content/drive/MyDrive/E資格機械学習プロジェクト/文章生成/output/generator_model_30.h5"

model = load_model(model_path)